# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/cities.csv")
del df["Unnamed: 0"]
df

,CITY,COUNTRY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED
0,Iquique,CL,-20.2208,-70.1431,60.84,60.84,65,22,5.26
1,Vostok,RU,46.4856,135.8833,64.29,64.29,99,64,4.54
2,Albany,US,42.6001,-73.9662,76.66,76.66,53,15,1.99
3,Puerto Ayora,EC,-0.7393,-90.3518,72.55,72.55,84,83,5.01
4,Banda Aceh,ID,5.5577,95.3222,80.40,80.40,76,99,12.08
5,Busselton,AU,-33.6500,115.3333,53.69,53.69,64,99,5.46
6,Barrow,US,71.2906,-156.7887,38.26,38.26,83,100,7.96
7,Jamestown,US,42.0970,-79.2353,72.77,72.77,56,40,11.50
8,Norman Wells,CA,65.2820,-126.8329,57.24,57.24,77,75,4.61
9,San Patricio,US,28.0170,-97.5169,93.06,93.06,40,33,15.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) 

In [4]:
locations = df[["LAT", "LNG"]]

weights = df["HUMIDITY"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
best_cities_df = df.loc[ (df["TEMP"].between(69, 84)) & (df["HUMIDITY"] < 80) & (df["CLOUDINESS"] < 50), :]
best_cities_df.dropna()
best_cities_df

,CITY,COUNTRY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED
2,Albany,US,42.6001,-73.9662,76.66,76.66,53,15,1.99
7,Jamestown,US,42.0970,-79.2353,72.77,72.77,56,40,11.50
13,Vukovar,HR,45.3433,18.9997,76.77,76.77,61,20,11.50
15,Madingou,CG,-4.1536,13.5500,82.22,82.22,50,40,10.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = best_cities_df.copy()

hotel_df["HOTEL_PLACEID"] = "NULL"
hotel_df["HOTEL_NAME"] = "NULL"
hotel_df["HOTEL_CITY"] = "NULL"
hotel_df["HOTEL_COUNTRY"] = "NULL"

hotel_df

# hotel_df = hotel_df.head(10)

,CITY,COUNTRY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
2,Albany,US,42.6001,-73.9662,76.66,76.66,53,15,1.99,NULL,NULL,NULL,NULL
7,Jamestown,US,42.0970,-79.2353,72.77,72.77,56,40,11.50,NULL,NULL,NULL,NULL
13,Vukovar,HR,45.3433,18.9997,76.77,76.77,61,20,11.50,NULL,NULL,NULL,NULL
15,Madingou,CG,-4.1536,13.5500,82.22,82.22,50,40,10.89,NULL,NULL,NULL,NULL


In [8]:
# params dictionary to update each iteration

params = {
    "location": "0,0",
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["LAT"]
    lng = row["LNG"]

    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_data = requests.get(base_url, params=params)

    # convert to json
    hotel_json = hotel_data.json()
    # pprint(hotel_json)

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.

    try:
        hotel_df.loc[index, "HOTEL_PLACEID"] = hotel_json["results"][0]["place_id"]
#         hotel_df.loc[index, "HOTEL_CITY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[0].split(" ", 1)[1]
#         hotel_df.loc[index, "HOTEL_COUNTRY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[1]
#         # print(f'HOTEL TO ADD: {hotel_json["results"][0]["name"]}')
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [14]:
# pprint(hotel_json)


In [15]:
cleanhotel_df = hotel_df.loc[ (hotel_df["HOTEL_PLACEID"] != "NULL"), :]

cleanhotel_df

# df = hotel_df.loc[   hotel_df["HOTEL_NAME"] != "NULL", :]
# df.reset_index(drop=True)
# df


,CITY,COUNTRY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
7,Jamestown,US,42.0970,-79.2353,72.77,72.77,56,40,11.50,ChIJ6zM4eV6p0okRE5a0-lRuihI,NULL,NULL,NULL
13,Vukovar,HR,45.3433,18.9997,76.77,76.77,61,20,11.50,ChIJ54Hhhux5W0cRfoB2qx9McBQ,NULL,NULL,NULL
15,Madingou,CG,-4.1536,13.5500,82.22,82.22,50,40,10.89,ChIJ34KpWXaCaBoR9oc1hv6eRGg,NULL,NULL,NULL


In [19]:
# params dictionary to update each iteration

params = {
    "place_id": "",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in cleanhotel_df.iterrows():
    
    place_id = row["HOTEL_PLACEID"]

    # change location each iteration while leaving original params in place
    
    params["place_id"] = f"{place_id}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"

    # make request and print url
    address_data = requests.get(base_url, params=params)

    # convert to json
    address_json = address_data.json()
    pprint(address_json)
    
    

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.

    try:
        cleanhotel_df.loc[index, "HOTEL_CITY"] = address_json["result"]["formatted_address"]
#         hotel_df.loc[index, "HOTEL_CITY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[0].split(" ", 1)[1]
#         hotel_df.loc[index, "HOTEL_COUNTRY"] = hotel_json["results"][0]["plus_code"]["compound_code"].split(",")[1]
#         # print(f'HOTEL TO ADD: {hotel_json["results"][0]["name"]}')
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '10',
                                    'short_name': '10',
                                    'types': ['street_number']},
                                   {'long_name': 'Dunham Avenue',
                                    'short_name': 'Dunham Ave',
                                    'types': ['route']},
                                   {'long_name': 'Celoron',
                                    'short_name': 'Celoron',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Ellicott',
                                    'short_name': 'Ellicott',
                                    'types': ['administrative_area_level_3',
                                              'political']},
                                   {'long_name': 'Chautauqua County',
                                    'short_name': 'Chautauqua County',
               

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '18',
                                    'short_name': '18',
                                    'types': ['street_number']},
                                   {'long_name': 'Ulica Josipa Jurja '
                                                 'Strossmayera',
                                    'short_name': 'Ul. Josipa Jurja '
                                                  'Strossmayera',
                                    'types': ['route']},
                                   {'long_name': 'Vukovar',
                                    'short_name': 'Vukovar',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Općina Vukovar',
                                    'short_name': 'Općina Vukovar',
                                    'types': ['administrative_area_level_2',
                                              'political']},
  

 'result': {'address_components': [{'long_name': 'RGJW+8H2',
                                    'short_name': 'RGJW+8H2',
                                    'types': ['plus_code']},
                                   {'long_name': 'Madingou',
                                    'short_name': 'Madingou',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Madingou',
                                    'short_name': 'Madingou',
                                    'types': ['administrative_area_level_2',
                                              'political']},
                                   {'long_name': 'Bouenza',
                                    'short_name': 'Bouenza',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_name': 'Republic of the Congo',
                          

In [20]:
cleanhotel_df

,CITY,COUNTRY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED,HOTEL_PLACEID,HOTEL_NAME,HOTEL_CITY,HOTEL_COUNTRY
7,Jamestown,US,42.0970,-79.2353,72.77,72.77,56,40,11.50,ChIJ6zM4eV6p0okRE5a0-lRuihI,NULL,"10 Dunham Ave, Celoron, NY 14720, USA",NULL
13,Vukovar,HR,45.3433,18.9997,76.77,76.77,61,20,11.50,ChIJ54Hhhux5W0cRfoB2qx9McBQ,NULL,"Ul. Josipa Jurja Strossmayera 18, 32000, Vukov...",NULL
15,Madingou,CG,-4.1536,13.5500,82.22,82.22,50,40,10.89,ChIJ34KpWXaCaBoR9oc1hv6eRGg,NULL,"RGJW+8H2, Madingou, Republic of the Congo",NULL


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{HOTEL_NAME}</dd>
<dt>City</dt><dd>{HOTEL_CITY}</dd>
<dt>Country</dt><dd>{HOTEL_COUNTRY}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["LAT", "LNG"]]

In [ ]:
# Add marker layer ontop of heat map

 # Assign the marker layer to a variable
    
markers = gmaps.marker_layer(locations=locations, display_info_box=True, info_box_content=hotel_info)

# Add the layer to the map

fig.add_layer(markers)
fig


# Display figure
